# Naive Models

The purpose of this notebook is to devise some naive models using our financial input data so that we have a comparison point for the predictive power of our TCN models. 

## Singular Gaussian

Our first Naive model will be a single gaussian whose mean and variance are calculated from the training data. We will then apply this model to the test data and calculate an average probability density and NLL for the test data.

In [10]:
import numpy as np
import pandas as pd

In [6]:
input_data = np.load("../Data/Input/Basic/input_data.pkl", allow_pickle=True) 
target_data = np.load("../Data/Input/Basic/input_target.pkl", allow_pickle=True)

In [14]:
enddate = pd.to_datetime("2019-01-01",format='%Y-%m-%d')

train_input_data = input_data[input_data.index < enddate]
train_target_data = target_data[target_data.index < enddate]

In [18]:
train_target_data

,relative_diff
Date,
2005-01-04,-0.163781
2005-01-05,-1.218655
2005-01-06,-0.735469
2005-01-07,0.262906
2005-01-10,0.619785
...,...
2018-12-24,-0.299950
2018-12-26,0.760488
2018-12-27,0.406403


In [28]:
train_mean = train_target_data["relative_diff"].mean() 
train_var = train_target_data["relative_diff"].var()

print(train_mean, train_var)

0.03646447553058314 1.365515330020188


In [34]:
startdate = pd.to_datetime("2019-01-01",format='%Y-%m-%d')
enddate = pd.to_datetime("2022-01-01",format='%Y-%m-%d')

test_input_data = input_data[input_data.index >= startdate]
test_target_data = target_data[target_data.index >= startdate]

test_input_data = test_input_data[test_input_data.index < enddate]
test_target_data = test_target_data[test_target_data.index < enddate]

In [36]:
test_target_data

,relative_diff
Date,
2019-01-02,0.906618
2019-01-03,-0.808625
2019-01-04,0.345850
2019-01-07,-0.270803
2019-01-08,0.641817
...,...
2021-12-27,-0.431009
2021-12-28,-0.029649
2021-12-29,0.717718


In [52]:
# Probability density based on single training gaussian

prob_density = (1.0/np.sqrt(2.0*np.pi*train_var))*np.exp(-0.5*(train_mean-test_target_data["relative_diff"])**2/train_var)

print(prob_density)
print()
print("The average probability density is: ",prob_density.values.mean())

Date
2019-01-02    0.258735
2019-01-03    0.262840
2019-01-04    0.329640
2019-01-07    0.329798
2019-01-08    0.298530
                ...   
2021-12-27    0.315145
2021-12-28    0.340853
2021-12-29    0.288044
2021-12-30    0.292925
2021-12-31    0.137680
Name: relative_diff, Length: 783, dtype: float64

The average probability density is:  0.2790070758008114


In [56]:
# NLL based on single training gaussian

NLL = 0.5*(np.log(train_var)+((train_mean-test_target_data["relative_diff"])**2/train_var))

print(NLL)
print()
print("The average NLL density is: ",NLL.values.mean())

Date
2019-01-02    0.433012
2019-01-03    0.417270
2019-01-04    0.190815
2019-01-07    0.190336
2019-01-08    0.289947
                ...   
2021-12-27    0.235784
2021-12-28    0.157366
2021-12-29    0.325704
2021-12-30    0.308899
2021-12-31    1.063888
Name: relative_diff, Length: 783, dtype: float64

The average NLL density is:  0.48560618291724555


In [66]:
# MAE based on single training gaussian

MAE = abs(train_mean-test_target_data)

print(MAE)
print()
print("The average MAE is: ",MAE.values.mean())

            relative_diff
Date                     
2019-01-02       0.870154
2019-01-03       0.845090
2019-01-04       0.309385
2019-01-07       0.307267
2019-01-08       0.605352
...                   ...
2021-12-27       0.467474
2021-12-28       0.066113
2021-12-29       0.681253
2021-12-30       0.646692
2021-12-31       1.574836

[783 rows x 1 columns]

The average MAE is:  0.6651051309738522


### Summary

Based on constructing a single Gaussian expectation from the training data and using this as a predictive tool we see that the average probability density was 0.279 and the average NLL (without a constant) was 0.485 (rounded down). Therefore any model that can score a higher average probability density than 0.279 and a lower NLL than 0.485 has some predictive power.

On a side note we see that the MAE using just the mean of the training targets is 0.665 so that any model with predictive power on the mean values alone will have a lower MAE than 0.665.

## Singular Gaussian from TCN

Our next naive model will actually use the TCN, we will therefore need to include it as an output to the training script. Our TCN model will use each sample of input data in the test set to predict a mean and variance for that samples target. 

What we want to know know is how much better these individual predictions are over having the network predict a singular Gaussian Does the TCN currently predict some more extreme values that either improve or detract from its overall performance, to answer this we will compare to the average probability density and NLL of a model that uses the average mean and variance of the outputs in a singular gaussian.



### Summary

Based on the output statistics below we can see that at the start of the training a singular gaussian prediction outperformed the model predictions for a few epochs, however, for the following epoch the model significantly outperformed a singular gaussian prediction demonstrating that it has complex predictive value.

In [ ]:
Using Apple MPS Framework for GPU acceleration with 1 local device/s 

Model initialised with 0.0100 million parameters occupying 26.78 MB 

Training begins... 

| Epoch   1 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  7.20 | loss 0.69024033 |
| Epoch   1 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.71 | loss 0.66073505 |
| Epoch   1 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.66 | loss 0.69168214 |
| Epoch   1 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  8.52 | loss 0.62779196 |

 Test statistics 

| Average Loss 0.44856 | Average Probability Density 0.31782 | Average MAE on mean 0.66405 |
| Average mean 0.05798 | Average absolute value of mean 0.05798 | Average variance 0.93799 |

| Singular Loss 0.44707 | Singular Probability Density 0.31763 | Singular MAE on mean 0.66373 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   2 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.23 | loss 0.62232863 |
| Epoch   2 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.31 | loss 0.66518204 |
| Epoch   2 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.50 | loss 0.58365321 |
| Epoch   2 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.31 | loss 0.66744639 |

 Test statistics 

| Average Loss 0.45331 | Average Probability Density 0.32642 | Average MAE on mean 0.66559 |
| Average mean 0.00562 | Average absolute value of mean 0.01005 | Average variance 0.85601 |

| Singular Loss 0.44823 | Singular Probability Density 0.32625 | Singular MAE on mean 0.66515 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   3 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.36 | loss 0.60475207 |
| Epoch   3 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.41 | loss 0.61542166 |
| Epoch   3 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  7.01 | loss 0.62081293 |
| Epoch   3 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.10 | loss 0.60496626 |

 Test statistics 

| Average Loss 0.45856 | Average Probability Density 0.33500 | Average MAE on mean 0.67040 |
| Average mean -0.03362 | Average absolute value of mean 0.03506 | Average variance 0.78199 |

| Singular Loss 0.45593 | Singular Probability Density 0.33438 | Singular MAE on mean 0.67045 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   4 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.30 | loss 0.64695081 |
| Epoch   4 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.70 | loss 0.54355725 |
| Epoch   4 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.77 | loss 0.58936955 |
| Epoch   4 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.82 | loss 0.60393532 |

 Test statistics 

| Average Loss 0.47023 | Average Probability Density 0.34981 | Average MAE on mean 0.66546 |
| Average mean 0.00164 | Average absolute value of mean 0.01530 | Average variance 0.68865 |

| Singular Loss 0.46754 | Singular Probability Density 0.34864 | Singular MAE on mean 0.66537 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   5 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.19 | loss 0.62364940 |
| Epoch   5 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.30 | loss 0.52898206 |
| Epoch   5 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.41 | loss 0.57938135 |
| Epoch   5 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.79 | loss 0.65810245 |

 Test statistics 

| Average Loss 0.43563 | Average Probability Density 0.32690 | Average MAE on mean 0.66661 |
| Average mean 0.02313 | Average absolute value of mean 0.05255 | Average variance 0.88822 |

| Singular Loss 0.44696 | Singular Probability Density 0.32280 | Singular MAE on mean 0.66435 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   6 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.26 | loss 0.56569330 |
| Epoch   6 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.91 | loss 0.55994462 |
| Epoch   6 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.43 | loss 0.58446950 |
| Epoch   6 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.25 | loss 0.58556090 |

 Test statistics 

| Average Loss 0.43494 | Average Probability Density 0.34550 | Average MAE on mean 0.66577 |
| Average mean 0.03698 | Average absolute value of mean 0.05789 | Average variance 0.74172 |

| Singular Loss 0.45648 | Singular Probability Density 0.34180 | Singular MAE on mean 0.66393 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   7 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.17 | loss 0.53003741 |
| Epoch   7 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.59 | loss 0.57023246 |
| Epoch   7 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.35 | loss 0.58107783 |
| Epoch   7 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.42 | loss 0.51277679 |

 Test statistics 

| Average Loss 0.47218 | Average Probability Density 0.36686 | Average MAE on mean 0.66713 |
| Average mean 0.00597 | Average absolute value of mean 0.04033 | Average variance 0.59151 |

| Singular Loss 0.49860 | Singular Probability Density 0.36451 | Singular MAE on mean 0.66513 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   8 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.20 | loss 0.55898335 |
| Epoch   8 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.38 | loss 0.53897083 |
| Epoch   8 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.50 | loss 0.56180387 |
| Epoch   8 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  7.05 | loss 0.54670331 |

 Test statistics 

| Average Loss 0.45513 | Average Probability Density 0.36289 | Average MAE on mean 0.67080 |
| Average mean -0.00846 | Average absolute value of mean 0.05420 | Average variance 0.61647 |

| Singular Loss 0.48961 | Singular Probability Density 0.35972 | Singular MAE on mean 0.66660 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch   9 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.06 | loss 0.52059090 |
| Epoch   9 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.29 | loss 0.54714915 |
| Epoch   9 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.62 | loss 0.58784461 |
| Epoch   9 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.42 | loss 0.55716258 |

 Test statistics 

| Average Loss 0.42779 | Average Probability Density 0.35349 | Average MAE on mean 0.66929 |
| Average mean 0.02955 | Average absolute value of mean 0.07188 | Average variance 0.69478 |

| Singular Loss 0.46489 | Singular Probability Density 0.34847 | Singular MAE on mean 0.66413 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  10 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.08 | loss 0.49531491 |
| Epoch  10 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.52 | loss 0.56308926 |
| Epoch  10 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.89 | loss 0.57642244 |
| Epoch  10 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.47 | loss 0.53874964 |

 Test statistics 

| Average Loss 0.43033 | Average Probability Density 0.35075 | Average MAE on mean 0.67286 |
| Average mean 0.00057 | Average absolute value of mean 0.07294 | Average variance 0.70742 |

| Singular Loss 0.46370 | Singular Probability Density 0.34583 | Singular MAE on mean 0.66543 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  11 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.37 | loss 0.58097862 |
| Epoch  11 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.42 | loss 0.51108057 |
| Epoch  11 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.09 | loss 0.54653051 |
| Epoch  11 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.10 | loss 0.47875446 |

 Test statistics 

| Average Loss 0.45699 | Average Probability Density 0.36803 | Average MAE on mean 0.66941 |
| Average mean 0.00975 | Average absolute value of mean 0.05630 | Average variance 0.59354 |

| Singular Loss 0.49745 | Singular Probability Density 0.36428 | Singular MAE on mean 0.66494 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  12 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.35 | loss 0.54617060 |
| Epoch  12 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.61 | loss 0.49716915 |
| Epoch  12 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.70 | loss 0.55023026 |
| Epoch  12 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.01 | loss 0.56491670 |

 Test statistics 

| Average Loss 0.42867 | Average Probability Density 0.36047 | Average MAE on mean 0.67119 |
| Average mean 0.03478 | Average absolute value of mean 0.08911 | Average variance 0.66405 |

| Singular Loss 0.47208 | Singular Probability Density 0.35330 | Singular MAE on mean 0.66398 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  13 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.30 | loss 0.52769579 |
| Epoch  13 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.50 | loss 0.51376726 |
| Epoch  13 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.14 | loss 0.55049450 |
| Epoch  13 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.21 | loss 0.51920547 |

 Test statistics 

| Average Loss 0.40552 | Average Probability Density 0.34692 | Average MAE on mean 0.67452 |
| Average mean -0.00107 | Average absolute value of mean 0.08711 | Average variance 0.76368 |

| Singular Loss 0.45516 | Singular Probability Density 0.33787 | Singular MAE on mean 0.66560 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  14 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.36 | loss 0.54658026 |
| Epoch  14 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.74 | loss 0.48782069 |
| Epoch  14 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.30 | loss 0.52498843 |
| Epoch  14 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.19 | loss 0.55610947 |

 Test statistics 

| Average Loss 0.42523 | Average Probability Density 0.35646 | Average MAE on mean 0.67452 |
| Average mean -0.00176 | Average absolute value of mean 0.08422 | Average variance 0.69293 |

| Singular Loss 0.46684 | Singular Probability Density 0.34790 | Singular MAE on mean 0.66569 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  15 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.27 | loss 0.50022503 |
| Epoch  15 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.70 | loss 0.48589208 |
| Epoch  15 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.55 | loss 0.58360509 |
| Epoch  15 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.57 | loss 0.50098881 |

 Test statistics 

| Average Loss 0.40134 | Average Probability Density 0.35148 | Average MAE on mean 0.67704 |
| Average mean 0.00328 | Average absolute value of mean 0.09404 | Average variance 0.74844 |

| Singular Loss 0.45682 | Singular Probability Density 0.34007 | Singular MAE on mean 0.66528 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  16 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.43 | loss 0.53511293 |
| Epoch  16 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.61 | loss 0.50288449 |
| Epoch  16 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.29 | loss 0.50822607 |
| Epoch  16 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.35 | loss 0.51649770 |

 Test statistics 

| Average Loss 0.40573 | Average Probability Density 0.35048 | Average MAE on mean 0.67683 |
| Average mean 0.06025 | Average absolute value of mean 0.12339 | Average variance 0.76943 |

| Singular Loss 0.45301 | Singular Probability Density 0.33831 | Singular MAE on mean 0.66376 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  17 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.46 | loss 0.51494738 |
| Epoch  17 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.31 | loss 0.51235300 |
| Epoch  17 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.25 | loss 0.50294393 |
| Epoch  17 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.07 | loss 0.54485624 |

 Test statistics 

| Average Loss 0.41440 | Average Probability Density 0.31193 | Average MAE on mean 0.68903 |
| Average mean 0.02383 | Average absolute value of mean 0.14695 | Average variance 1.17626 |

| Singular Loss 0.46342 | Singular Probability Density 0.29403 | Singular MAE on mean 0.66432 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  18 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.47 | loss 0.50262060 |
| Epoch  18 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.32 | loss 0.51104112 |
| Epoch  18 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.55 | loss 0.51504399 |
| Epoch  18 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.17 | loss 0.44759380 |

 Test statistics 

| Average Loss 0.42161 | Average Probability Density 0.36402 | Average MAE on mean 0.67126 |
| Average mean 0.04467 | Average absolute value of mean 0.09611 | Average variance 0.65436 |

| Singular Loss 0.47462 | Singular Probability Density 0.35504 | Singular MAE on mean 0.66379 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  19 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.48 | loss 0.53578071 |
| Epoch  19 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.23 | loss 0.50980918 |
| Epoch  19 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.40 | loss 0.44965206 |
| Epoch  19 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.28 | loss 0.51536867 |

 Test statistics 

| Average Loss 0.43131 | Average Probability Density 0.36195 | Average MAE on mean 0.67417 |
| Average mean 0.02215 | Average absolute value of mean 0.09392 | Average variance 0.65882 |

| Singular Loss 0.47388 | Singular Probability Density 0.35383 | Singular MAE on mean 0.66439 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  20 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.37 | loss 0.47857649 |
| Epoch  20 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.48 | loss 0.47944754 |
| Epoch  20 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.40 | loss 0.52723896 |
| Epoch  20 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.30 | loss 0.47391109 |

 Test statistics 

| Average Loss 0.43274 | Average Probability Density 0.36561 | Average MAE on mean 0.67827 |
| Average mean 0.00590 | Average absolute value of mean 0.08627 | Average variance 0.63595 |

| Singular Loss 0.48164 | Singular Probability Density 0.35702 | Singular MAE on mean 0.66513 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  21 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.51 | loss 0.49783786 |
| Epoch  21 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.42 | loss 0.49276887 |
| Epoch  21 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.26 | loss 0.49950366 |
| Epoch  21 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.29 | loss 0.45646685 |

 Test statistics 

| Average Loss 0.44918 | Average Probability Density 0.36935 | Average MAE on mean 0.67640 |
| Average mean 0.01144 | Average absolute value of mean 0.08682 | Average variance 0.60527 |

| Singular Loss 0.49243 | Singular Probability Density 0.36231 | Singular MAE on mean 0.66486 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  22 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.22 | loss 0.41338970 |
| Epoch  22 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.28 | loss 0.49058450 |
| Epoch  22 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.16 | loss 0.55324692 |
| Epoch  22 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.91 | loss 0.50754554 |

 Test statistics 

| Average Loss 0.42387 | Average Probability Density 0.35291 | Average MAE on mean 0.68890 |
| Average mean -0.00468 | Average absolute value of mean 0.12354 | Average variance 0.74676 |

| Singular Loss 0.45757 | Singular Probability Density 0.34008 | Singular MAE on mean 0.66607 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  23 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.15 | loss 0.50313828 |
| Epoch  23 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.41 | loss 0.47623724 |
| Epoch  23 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.21 | loss 0.48332951 |
| Epoch  23 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.09 | loss 0.49844003 |

 Test statistics 

| Average Loss 0.42613 | Average Probability Density 0.35291 | Average MAE on mean 0.68977 |
| Average mean -0.00069 | Average absolute value of mean 0.12494 | Average variance 0.74711 |

| Singular Loss 0.45725 | Singular Probability Density 0.34015 | Singular MAE on mean 0.66555 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  24 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.26 | loss 0.44921538 |
| Epoch  24 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.56 | loss 0.51024289 |
| Epoch  24 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.16 | loss 0.49745005 |
| Epoch  24 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.46 | loss 0.49806901 |

 Test statistics 

| Average Loss 0.42449 | Average Probability Density 0.34559 | Average MAE on mean 0.69592 |
| Average mean -0.03536 | Average absolute value of mean 0.14119 | Average variance 0.80585 |

| Singular Loss 0.45400 | Singular Probability Density 0.33125 | Singular MAE on mean 0.67074 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  25 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.13 | loss 0.45882103 |
| Epoch  25 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.32 | loss 0.49136388 |
| Epoch  25 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.26 | loss 0.48147108 |
| Epoch  25 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.09 | loss 0.57719009 |

 Test statistics 

| Average Loss 0.42285 | Average Probability Density 0.34412 | Average MAE on mean 0.69802 |
| Average mean -0.05970 | Average absolute value of mean 0.13953 | Average variance 0.80264 |

| Singular Loss 0.45715 | Singular Probability Density 0.33072 | Singular MAE on mean 0.67521 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  26 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.28 | loss 0.44333463 |
| Epoch  26 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.41 | loss 0.48587861 |
| Epoch  26 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.30 | loss 0.53945843 |
| Epoch  26 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.13 | loss 0.41869196 |

 Test statistics 

| Average Loss 0.44778 | Average Probability Density 0.35926 | Average MAE on mean 0.68625 |
| Average mean 0.03377 | Average absolute value of mean 0.14002 | Average variance 0.71635 |

| Singular Loss 0.46059 | Singular Probability Density 0.34537 | Singular MAE on mean 0.66400 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  27 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.22 | loss 0.52252977 |
| Epoch  27 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.67 | loss 0.47533175 |
| Epoch  27 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.31 | loss 0.47881223 |
| Epoch  27 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.17 | loss 0.39385530 |

 Test statistics 

| Average Loss 0.47182 | Average Probability Density 0.37507 | Average MAE on mean 0.67746 |
| Average mean 0.03488 | Average absolute value of mean 0.09693 | Average variance 0.57406 |

| Singular Loss 0.50536 | Singular Probability Density 0.36849 | Singular MAE on mean 0.66398 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  28 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.17 | loss 0.47666302 |
| Epoch  28 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.32 | loss 0.46276800 |
| Epoch  28 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.15 | loss 0.51035424 |
| Epoch  28 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.18 | loss 0.49512355 |

 Test statistics 

| Average Loss 0.46644 | Average Probability Density 0.36867 | Average MAE on mean 0.68059 |
| Average mean 0.01300 | Average absolute value of mean 0.10474 | Average variance 0.61104 |

| Singular Loss 0.49007 | Singular Probability Density 0.36138 | Singular MAE on mean 0.66478 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  29 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.14 | loss 0.48469257 |
| Epoch  29 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.20 | loss 0.47951658 |
| Epoch  29 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.42 | loss 0.46208709 |
| Epoch  29 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.28 | loss 0.50649284 |

 Test statistics 

| Average Loss 0.43661 | Average Probability Density 0.36287 | Average MAE on mean 0.68127 |
| Average mean 0.01405 | Average absolute value of mean 0.10709 | Average variance 0.66088 |

| Singular Loss 0.47368 | Singular Probability Density 0.35328 | Singular MAE on mean 0.66473 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl

| Epoch  30 |   128/  428 batches |  1024/ 3422 samples | lr 0.00050 | ms/batch  6.16 | loss 0.50715359 |
| Epoch  30 |   256/  428 batches |  2048/ 3422 samples | lr 0.00050 | ms/batch  6.52 | loss 0.43903594 |
| Epoch  30 |   384/  428 batches |  3072/ 3422 samples | lr 0.00050 | ms/batch  6.55 | loss 0.49081773 |
| Epoch  30 |   428/  428 batches |  3422/ 3422 samples | lr 0.00050 | ms/batch  6.37 | loss 0.48358516 |

 Test statistics 

| Average Loss 0.42455 | Average Probability Density 0.34881 | Average MAE on mean 0.69358 |
| Average mean 0.00201 | Average absolute value of mean 0.14293 | Average variance 0.80352 |

| Singular Loss 0.45115 | Singular Probability Density 0.33270 | Singular MAE on mean 0.66535 |

Model saved to path: outputs/regularization/singular_test/regularization.pkl